# Cifar 10 integration test

## Configuration

In [1]:
import os
from datetime import datetime

import numpy as np
import tensorflow as tf


FILE_PATH = os.getcwd()
MODEL_PATH = os.path.join(FILE_PATH, "../models/my_vgg.h5")

## Cifar 10 model

### Model

In [2]:
from tensorflow.keras.regularizers import l2


def my_vgg():
    model = tf.keras.Sequential(
        [
            tf.keras.layers.Conv2D(
                32,
                (3, 3),
                activation="relu",
                kernel_initializer="he_uniform",
                kernel_regularizer=l2(0.001),
                padding="same",
                input_shape=(32, 32, 3),
            ),
            tf.keras.layers.Conv2D(
                32,
                (3, 3),
                activation="relu",
                kernel_initializer="he_uniform",
                kernel_regularizer=l2(0.001),
                padding="same",
            ),
            tf.keras.layers.MaxPooling2D(2, 2),
            tf.keras.layers.Conv2D(
                64,
                (3, 3),
                activation="relu",
                kernel_initializer="he_uniform",
                kernel_regularizer=l2(0.001),
                padding="same",
            ),
            tf.keras.layers.Conv2D(
                64,
                (3, 3),
                activation="relu",
                kernel_initializer="he_uniform",
                kernel_regularizer=l2(0.001),
                padding="same",
            ),
            tf.keras.layers.MaxPooling2D(2, 2),
            tf.keras.layers.Conv2D(
                128,
                (3, 3),
                activation="relu",
                kernel_initializer="he_uniform",
                kernel_regularizer=l2(0.001),
                padding="same",
            ),
            tf.keras.layers.Conv2D(
                128,
                (3, 3),
                activation="relu",
                kernel_initializer="he_uniform",
                kernel_regularizer=l2(0.001),
                padding="same",
            ),
            tf.keras.layers.MaxPooling2D(2, 2),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(
                128,
                activation="relu",
                kernel_initializer="he_uniform",
                kernel_regularizer=l2(0.001),
            ),
            tf.keras.layers.Dense(10, activation="softmax"),
        ]
    )

    model.compile(
        optimizer="adam",
        loss=tf.losses.CategoricalCrossentropy(from_logits=True),
        metrics=[tf.metrics.CategoricalAccuracy()],
    )
    return model


### Training

In [3]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

if os.path.exists(MODEL_PATH):
    model: tf.keras.Model = tf.keras.models.load_model(MODEL_PATH)
else:
    model: tf.keras.Model = my_vgg()
    model.fit(
        x_train,
        y_train,
        epochs=100,
        batch_size=64,
        validation_data=(x_test, y_test),
    )
    model.save(MODEL_PATH)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        3

### Evaluation

In [4]:
loss, acc = model.evaluate(x_test, y_test)

print(f"loss={loss}, acc={acc}")

313/313 [==============================] - 5s 7ms/step - loss: 0.8136 - categorical_accuracy: 0.8362
loss=0.8136339783668518, acc=0.8361999988555908


## Fault Injection

### Model

In [5]:
from inputtensorfi import InputTensorFI
from inputtensorfi.layers import PixelBitFiLayerTF
from inputtensorfi.manipulation.bit.action import BitFlip
from inputtensorfi.manipulation.img.faults import BitFault

bit_faults = np.array(
    [
        BitFault(5, 5, 0, 7, BitFlip),
        BitFault(6, 6, 0, 7, BitFlip),
        BitFault(7, 7, 0, 7, BitFlip),
    ],
    dtype=object,
)

faulted_model = InputTensorFI.build_faulted_model(
    model,
    fi_layers=[
        PixelBitFiLayerTF(bit_faults, dtype=tf.uint8),
    ],
)
faulted_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
pixel_bit_fi_layer_tf (Pixel (None, 32, 32, 3)         0         
_________________________________________________________________
sequential (Sequential)      (None, 10)                698154    
Total params: 698,154
Trainable params: 698,154
Non-trainable params: 0
_________________________________________________________________


### Evaluation

We are going to trace the model for debug purpose. These lines are optionals.

In [6]:
logdir = "logs/compile/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

Run the evaluation.

In [7]:
loss, acc = faulted_model.evaluate(
    x_test, y_test, callbacks=[tensorboard_callback]
)
print(f"loss={loss}, acc={acc}")

313/313 [==============================] - 14s 41ms/step - loss: 0.8240 - categorical_accuracy: 0.8310
loss=0.8326299786567688, acc=0.8310999870300293
